### Universidad Nacional de Lujan - Bases de Datos Masivas (11088) - Cavasin Nicolas #143501
# TP05-02 - Clustering

### Ejercicio 4:
**K-means**. Se provee un dataset sobre las características internas del núcleo de tres clases de trigo diferentes. Cargue el dataset en una de las herramientas de minería de datos provistas y resuelva:

a. Utilice el algoritmo k-medias variando la cantidad de centroides a efectos de agrupar los datos de la manera más eficiente.

b. ¿Cuál es la cantidad de grupos que permite un mejor agrupamiento de los datos? ¿Mediante cual métrica puede verificar esto?

c. ¿Cuáles son las características más distintivas de cada uno de los cluters resultantes?



In [1]:
! rm trigo.csv
!wget https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0502/trigo.csv

--2020-11-18 18:39:13--  https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0502/trigo.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.216.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.216.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9369 (9.1K) [text/plain]
Saving to: 'trigo.csv'

trigo.csv           100%[===================>]   9.15K  --.-KB/s    in 0.001s  

2020-11-18 18:39:14 (7.95 MB/s) - 'trigo.csv' saved [9369/9369]



In [2]:
import pandas as pd

trigo = pd.read_csv('trigo.csv', delimiter=';')

print(f'Cantidad de tuplas: {trigo.shape[0]}.')
print(f'Cantidad de columnas: {trigo.shape[1]}.')
print(f'Cantidad de NaN\'s: {trigo.isnull().sum().sum()}.\n')
print(f'Tipos de dato en cada columna: \n{trigo.dtypes}')
trigo.head()

Cantidad de tuplas: 210.
Cantidad de columnas: 8.
Cantidad de NaN's: 0.

Tipos de dato en cada columna: 
area               object
perimetro          object
compact            object
long_kernel        object
ancho_kernel       object
coef_asimetrima    object
long_ranura        object
tipo                int64
dtype: object


,area,perimetro,compact,long_kernel,ancho_kernel,coef_asimetrima,long_ranura,tipo
0,"15,26","14,84","0,871","5,763","3,312","2,221","5,22",1
1,"14,88","14,57","0,8811","5,554","3,333","1,018","4,956",1
2,"14,29","14,09","0,905","5,291","3,337","2,699","4,825",1
3,"13,84","13,94","0,8955","5,324","3,379","2,259","4,805",1
4,"16,14","14,99","0,9034","5,658","3,562","1,355","5,175",1


Se puede observar lo siguiente:

- El dataset no posee NaN's.
- Los valores son strings que deben ser convertidos a numericos.
- Posteriormente deben ser escalados para poder aplicar K-Means, ya que se basa en distancias y cada columna tiene diferentes unidades acordes al tipo de dato que representan.

In [3]:
import numpy as np

# Clono el ds
trigo_numerizado = trigo.copy()

# Convierto todos los strings a numericos
for col in trigo.columns:
    if col != 'tipo':
        trigo_numerizado[col] = pd.to_numeric(trigo[col].str.replace(',','.'))

# Muestro como quedo
trigo_numerizado.head()


,area,perimetro,compact,long_kernel,ancho_kernel,coef_asimetrima,long_ranura,tipo
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [4]:
# Ahora procedo con el escalado de valores
from sklearn.preprocessing import scale

# Escalo
scaled_1 = pd.DataFrame(scale(trigo_numerizado))

# Re inserto nombres de columnas
scaled_1.columns = trigo.columns

# Muestro resultado
scaled_1.head()

,area,perimetro,compact,long_kernel,ancho_kernel,coef_asimetrima,long_ranura,tipo
0,0.142098,0.215462,0.000061,0.304218,0.141702,-0.986152,-0.383577,-1.224745
1,0.011188,0.008224,0.428515,-0.168625,0.197432,-1.788166,-0.922013,-1.224745
2,-0.192067,-0.360201,1.442383,-0.763637,0.208048,-0.667479,-1.189192,-1.224745
3,-0.347091,-0.475333,1.039381,-0.688978,0.319508,-0.960818,-1.229983,-1.224745
4,0.445257,0.330595,1.374509,0.066666,0.805159,-1.563495,-0.475356,-1.224745


In [18]:
# Importo para clusterizar
from sklearn.cluster import KMeans

# Importo para graficar
import matplotlib.pyplot as plt

# Instancio el algoritmo con 5 centroides iniciales
km = KMeans(
    init='random',
    n_clusters=5,
    n_init=10, 
    random_state=0
)

# Ejecuto el algoritmo
y_km = km.fit_predict(scaled_1)

print(f'Agrupacion resultante:\n{y_km}')
print(f'ACAAA{scaled_1[y_km == 0]}')

plt.figure(figsize=(10,7))

#scatter del primer cluster
plt.scatter(
    scaled_1[y_km == 0, 0], scaled_1[y_km == 0, 1],
    s=50, c='lightgreen',
    marker='s', edgecolor='black',
    label='cluster 1'
)
#scatter del segundo cluster
plt.scatter(
    scaled_1[y_km == 1, 0], scaled_1[y_km == 1, 1],
    s=50, c='orange',
    marker='o', edgecolor='black',
    label='cluster 2'
)
#scatter del tercer cluster
plt.scatter(
    scaled_1[y_km == 2, 0], scaled_1[y_km == 2, 1],
    s=50, c='lightblue',
    marker='v', edgecolor='black',
    label='cluster 3'
)
#scatter del los centroides
plt.scatter(
    km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
    s=250, marker='*',
    c='red', edgecolor='black',
    label='centroides'
)
#le pongo la leyenda
plt.legend(scatterpoints=1)
#hace una grilla en el grafico
plt.grid()
#lo imprime en pantalla
plt.show()

Agrupacion resultante:
[0 0 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3
 3 0 0 0 0 0 3 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2
 3 3 3 2 2 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 3 2 2 2 2 2 2 3 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 3 2 3 2 2 2 2 3 2 2 3 3 3 3 2 3 3 3 1 1 1 4 1 1 4 4
 4 1 1 1 1 4 4 1 4 1 1 1 4 4 1 4 1 4 1 4 1 4 1 1 4 1 1 4 1 1 1 4 4 4 4 4 1
 1 1 4 1 4 1 4 4 1 4 4 4 4 4 4 4 4 4 1 4 4 4 1 4 4]
ACAAA        area  perimetro   compact  long_kernel  ancho_kernel  coef_asimetrima  \
0   0.142098   0.215462  0.000061     0.304218      0.141702        -0.986152   
1   0.011188   0.008224  0.428515    -0.168625      0.197432        -1.788166   
2  -0.192067  -0.360201  1.442383    -0.763637      0.208048        -0.667479   
3  -0.347091  -0.475333  1.039381    -0.688978      0.319508        -0.960818   
4   0.445257   0.330595  1.374509     0.066666      0.805159        -1.563495   
..       ...        ...       ...          ...           ..

TypeError: '(array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       